In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Load datasets
train = pd.read_csv('/kaggle/input/qtdt-project-ii/train.csv')
test = pd.read_csv('/kaggle/input/qtdt-project-ii/test.csv')
submission = pd.read_csv('/kaggle/input/qtdt-project-ii/sample_submission.csv')

# Identify columns
target = 'Response'
ID = 'id'
features = [col for col in train.columns if col not in [ID, target]]
cat_features = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']

# Split train into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
    train[features], train[target],
    test_size=0.2, random_state=42, stratify=train[target]
)

# Create Pools for CatBoost
train_pool = Pool(X_train, y_train, cat_features=cat_features)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_features)

# Initialize and train model

# Predict on test set

In [ ]:
model = CatBoostClassifier(
    iterations=6000,
    learning_rate=0.05,
    depth=6,
    eval_metric='AUC',
    cat_features=cat_features,
    early_stopping_rounds=50,
    verbose=100,
    random_seed=42
)

model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

# Evaluate on validation set
valid_preds = model.predict_proba(X_valid)[:, 1]
auc = roc_auc_score(y_valid, valid_preds)
print(f'Validation AUC: {auc:.4f}')


In [ ]:

test_preds = model.predict_proba(test[features])[:, 1]
submission['Response'] = test_preds

# Save submission file
submission.to_csv('submission.csv', index=False)
print("Submission file saved as submission.csv")

In [ ]:
re = pd.read_csv('/kaggle/working/submission.csv')
re.info()